In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# Parser

> Functionalities to parse the different information comming from pubmed results

In [ ]:
#| default_exp parser

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from datetime import datetime, timedelta, date
from collections import defaultdict, Counter
import re



In [ ]:
#| export
from fastcore.all import *
from pubmed_lib.author import Autor



In [ ]:
#| export
#| hide
regex = re.compile(("([a-z0-9!#$%&'*+\/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+\/=?^_`"
                    "{|}~-]+)*(@|\sat\s)(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?(\.|"
                    "\sdot\s))+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?)"))
reg_email = re.compile("[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$")
db_name = ''

In [ ]:
#| export
def parsePubmedData(
                        pubmeddata:dict #Data retrieve from pubmedData
                   ) ->list :
    """
    Receive the xml section of PubmedData and return list of ids
    """
    ids = {x.attributes['IdType']: str(x) for x in pubmeddata['ArticleIdList']}
    return ids

In [ ]:
#| export
def parseArticle(article_info:dict #Dictionary containing information of the article ('Article' Key)
                )->dict: #Somethign
    """
    Function to extract information from an Article, including Title, Abstract, AutorList, journal and published Date
    :param article_info: dictionary from key Article of an Medline citation
    :return (dict): tuple of dictionary with information from paper and autors
    """
    # Extract information about paper content
    title = article_info['ArticleTitle']
    journal = article_info['Journal']['Title']
    published_date = article_info['Journal']['JournalIssue']['PubDate']
    if 'ArticleDate' in article_info.keys() and len(article_info['ArticleDate'])>0:
        date_dict = {k.lower():int(v) for k,v in article_info['ArticleDate'][0].items()}
        published = date(**date_dict)
    elif 'Year' in published_date:
        year = published_date['Year']
        published = date(int(year),1,1)
    elif 'MedlineDate' in published_date:
        try:
            published = date(re.findall(r'\d\d\d\d',published_date['MedlineDate'])[0],1,1)
        except:
            published = date(published_date['MedlineDate'][:4],1,1)
    try:
        abstract = '. '.join(article_info['Abstract']['AbstractText'])
    except:
        abstract = ''
    try:
        autorlist = article_info['AuthorList']
    except:
        print('no autors found, jumping next')
        autorlist = []
    return {'abstract': abstract, 'autorlist': autorlist, 'title': title, 'journal': journal,
            'published':published}

In [ ]:
#| export
def parse_email(affil_text:str #Affiliations information for each autor in the Article
               )->str:

    """
    Find email from the Affiliation text (Maybe use the reg_email regex and not the one in the function, TEST)
    """
    match = re.search(reg_email , affil_text) #r'[\w.-]+@[\w.-]+', affil_text)
    if match is not None:
        email = match.group()
        email = email.strip('.;,')
    else:
        email = ''
    return email

In [ ]:
#| hide
# Function not being in use, I will leave it here for the moment until I am sure that delete it won't generate a bug
# def parseMayorKeys(citationInfo:dict, #Dictionary containing Keywords and Mesh info
#                   mesh:bool =False #If the Mesh Keywords would be included
#                   )->list:
#     """
#     Function to retrieve all keys from a citationInfo of an Article.
#     """
#     keywordList = citationInfo['KeywordList']
#     (mayorMesh, minorMesh) = parseMeshKeys(citationInfo) if mesh else ([],[])
#     mesh = [*mayorMesh, *minorMesh]
#     keys = [str(x) for x in keywordList[0] if x.attributes['MajorTopicYN'] == 'Y']
#     keys.extend(mayorMesh)
#     keys.extend(minorMesh)
#     return keys

In [ ]:
#| export
def parseMayorKeys(citationInfo:dict #Dictionary from 'MedlineCitation' key containing keywords information
                  )->list:
    """
    Function to Parse the Mayor Keys from the Article.
    """
    keywordList = citationInfo['KeywordList']
    if len(keywordList) == 0:
        return []
    else:
        return [str(x) for x in keywordList[0] if x.attributes['MajorTopicYN'] == 'Y']

In [ ]:
#| export
def parseMeshKeys(citationInfo:dict #Dictionary from 'MedlineCitation' key containing keywords information
                 )->(list,list):
    """
    Function to Parse the Mayor and Minor MeSH Keys from the Article.
    """
    meshKeys = citationInfo['MeshHeadingList'] if 'MeshHeadingList' in citationInfo.keys() else []
    mayorkeys = [str(x['DescriptorName']) for x in meshKeys if x['DescriptorName'].attributes['MajorTopicYN']=='Y']
    minorKeys = [str(x['DescriptorName']) for x in meshKeys if x['DescriptorName'].attributes['MajorTopicYN']=='N']
    return mayorkeys, minorKeys

In [ ]:
#| export
def parseKeys(
    citationInfo:dict, #Dictionary from 'MedlineCitation' key containing keywords information
)->tuple[list]:
    """
    Function to retrieve the mayor keys as well as MEsh Keys
    """
    return parseMayorKeys(citationInfo), parseMeshKeys(citationInfo)

In [ ]:
#| export 
def parse_paperinfo(
    paperinfo_xml:str #Xml data retreived from pubmed
    )->dict:
    """
    Function that parse the paper inforation and retrieve a dictionary 
    """
    PubmedData = parsePubmedData(paperinfo_xml['PubmedData'])
    article_xml = parseArticle(paperinfo_xml['MedlineCitation']['Article'])
    mayorKeys, (mayorMeshKeys, minorMeshKeys) = parseKeys(paperinfo_xml['MedlineCitation'])
    article_xml['mayorKeys'] = mayorKeys
    article_xml['mayorMesh'] = mayorMeshKeys
    article_xml['minorMesh'] = minorMeshKeys
    autorlist = []
    try:
        for author_xml in article_xml['autorlist']:
            if author_xml.attributes['ValidYN'] == 'N':
                continue
            autor_dict = parse_author_xml(author_xml)
            if autor_dict is None:
                continue
            autorlist.append(autor_dict)
    except:
        print('ERROR: parsing author {}'.format(author_xml))
    finally:
        article_xml['autorlist'] = autorlist
        PubmedData.update(article_xml)
    return PubmedData
    

In [ ]:
#| export
def parse_author_xml(
    autor_xml, #Xml data containing information for each author
    )->Autor:
    """
    Receive a dictionary from pubmed with the information of the Author. Retreive a Autor object with all the information parsed

    """
    # Return false if no author information found
    if 'CollectiveName' in autor_xml:
        return
    # try to parse information from XML
    try:
        #get Identifier (only orcid is used now so if they have identifier it should be the first value
        if len(autor_xml['Identifier']) > 0:
            autorID = str(autor_xml['Identifier'][0])
        else:
            autorID = ''
        #Get the affilaition details from that author, if he had
        if len(autor_xml['AffiliationInfo']) > 0:
            AFFs = ';'.join([affiliationinfo['Affiliation'] for affiliationinfo in autor_xml['AffiliationInfo']])
        else:
            AFFs = ''
        #Retrieving the name information, it is a must and should exist
        autorFN = autor_xml['ForeName']
        autorLN = autor_xml['LastName']
        autorIN = autor_xml['Initials']
        name = autorFN + ' ' + autorLN
        #Need to parse affiliation to get more details

        emails = parse_email(AFFs)
        data = {
            'Fname': autorFN,
            'Lname': autorLN,
            'emails': emails,
            'affiliations': AFFs, 
            'identifier': autorID,
            'name': name, 
            'initials': autorIN
               }  #
        # data.update(affilparsed)
        return Autor.parse_obj(data)

    except ValueError as e:
        print('not possible to get info value error')
        print(e)
        return
    except OSError as err:
        print("OS Error: {0}".format(err))
        return
    except:
        print('error en parsing')
        return

In [ ]:
#| hide
def find_country(address, country):
    """
    Find country from string
    """
    country = country.lower()
    state = ''
    for countries in COUNTRY:
        for c in countries:
            if c == country:
                country = countries[0]
                # print(country)
                state = find_state(address, country) if country in COUNTRIES_L.keys() else ''
    return country.title(), state

In [ ]:
#| hide
def find_state(address, country):
    """
    (str)->str
    Find state of Brazl from the affiliation details
    :param location:
    :return:
    """
    state_set = set(COUNTRIES_L[country])
    for state in state_set:
        if state in address.replace(',','').split(' '):
             return state.lstrip(' ')
    
    return ''

In [ ]:
#| hide
# def affilparser(text):
#     parsed_affil = affparser.parse(text)
#     return affparser.tuple2dict( parsed_affil)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()